This notebook is intended to investigate the methods of getting VJPs in different AD packages - namely Autograd, TensorFlow, and PyTorch, and clarify the behaviors of the relevant functions implemented in these packages. In cases where we are only interested in the gradient of the scalar loss function with regards to parameters, it is not necessary to explicitly build the intermediate VJPs as we can always get the VJP at any nodes by asking the automatic differentiator to calculate the gradient of the loss function with regards to those particular parameters. However, it can become essential if we want to calculate the product between a vector-vector Jacobian and an arbitrary vector which is not the derivative of any scalar node in the graph (that is not something like `dL / dv`) - say, in Gauss-Newton optimization, where we need to calculate the product between a Jacobian and a step vector. Moreover, writing the backpropagation process as a chain of VJPs provides insight for one to understand how things really work. We will assume readers have basic knowledge on the mathematical definitions of gradient, Jacobian matrix, Hessian matrix, and preferably, on the basic principle of backpropagation. 

The Gauss-Newton optimization codes by Saugat Kandel (https://github.com/saugatkandel/sopt) were heavily used as a reference when I tried to figure out how the `make_vjp` function in Autograd works. If you are looking for a working implementation of second-order optimization algorithms, please check out the cited repo. 

We will consider a simple model of `h(x) = y2(y1(x)) = exp(x^2)`, that is, `y2(g) = exp(g)` and `y1(g) = g^2`. The loss function `L = ||h(x) - y||^2`.

Now consider an input vector `x = [1, 2]`. The prediction function should give an output vector of `h = [2.71828183, 54.59815003]`. Suppose `y = [9.48773584, 518.01282467]` (which is the output that would be given by `x = [1.5, 2.5]`), the value of the scalar loss function should be `L = 214798.98617481123`.

Analytically, the Jacobian of `L` with regards to `h` for a 2-D vector input is
```
J_L_h = [2 * (h1 - y1), 2 * (h2 - y2)] = [-13.53890802, -926.82934927]
```
(note that `h` is also `f2`). Since `L` is a scalar, the Jacobian of `L` with regards to `h` is a vector. Although not entirely proper to write in this way, we can slightly abuse the notation and say
```
J_L_h ?= dL / dh = dL / df2.
```
The Jacobian of `h` (or `f2`) with reagrds to `f1` is
```
J_f2_f1 = [[exp(x1^2), 0        ],
            0        , exp(x2^2)]]
       ?= df2 / df1.
```
The VJP of `L` at `f1`, is then
```
VJP_L_f1 = J_f2_f1 * J_L_h = [-3.68025676e+01, -5.06031679e+04].
```
Again with the abuse of notation, we can recognize the nature of `VJP_f1` more clearly as
```
VJP_L_f1 = dL / df1.
```
In many literatures, it is more commonly written as `f1` with a dot on the top.

Similarly, the Jacobian of `f1` with regards to `x` is
```
J_f1_x = [[2 * x_1, 0       ],
          [0      , 2 * x_2 ]]
      ?= df1 / dx. 
```
and the VJP at `x` is then
```
VJP_L_x = J_f1_x * VJP_x
        = [-7.36051353e+01, -2.02412671e+05]
       ?= dL / dx.
```
That is, `VJP_x` gives the gradient of `L` with regards to the input vector `x`.

Lastly, we may also be interested in the Hessian of `L` with regards to `h`, which is crucial for implementing Gauss-Newton-based second-order optimization algorithms. The closed-form expression is rather simple for a least-square type of loss function:
```
d^2L / dh^2 = diag(2)
```
so the Hessian-vector-product (HVP) of any vector `v` with it would be `2v`. 

To summarize, numerically we have these VJPs:
```
dL / dh = [-13.53890802, -926.82934927]
dL / df1 = [-3.68025676e+01, -5.06031679e+04]
dL / dx = [-7.36051353e+01, -2.02412671e+05]
```

Let us create this model in Autograd first, and use the `make_vjp` function to get the intermediate VJPs with cross-validation with the above values that we have derived by hand.

## Autograd

### First-order operators

Let us define all the intermediate functions, and the final loss below:

In [52]:
import autograd.numpy as np
from autograd import grad

f1 = lambda g: g ** 2
f2 = lambda g: np.exp(g)
h = lambda x: f2(f1(x))
L = lambda h, y: np.sum((h - y) ** 2) 
L_x = lambda x, y: np.sum((h(x) - y) ** 2) 

Note that we have created two seemingly redundant instances of loss function: one takes the prediction functiuon `h` as input, and the other takes `x`. This difference matters because the input node determines "with regards to whom the VJP should be calculated". Now if we feed in the values of `x` and `y`, we get the supposed output values:

In [53]:
x = np.array([1., 2.])
y = np.array([9.48773584, 518.01282467])
print('Loss is ', L_x(x, y))

Loss is  214798.98617639724


The `grad` function of Autograd takes in the loss function, and return a function that has the same arguments as the loss, but returns the gradient instead. The second argument of `grad` is used to specify the variable that the gradient should be calculated for. 

In [54]:
grad_L_x_func = grad(L_x, [0])
J_L_x = grad_L_x_func(x, y)
print('Gradient dL / dx is ', dLdx[0])

grad_L_h_func = grad(L, [0])
J_L_h = grad_L_h_func(h(x), y)
print('Gradient dL / dh is ', j_L_h[0])

Gradient dL / dx is  [-7.36051353e+01 -2.02412671e+05]
Gradient dL / dh is  [ -13.53890802 -926.82934927]


These agree with our hand-derived solutions! But sometimes we also want to know the intermediate Jacobians and VJPs - in particular, the Jacobian of `h` at `x` (which gives `dh / dx`), which is essential when using a Gauss-Newton type of second-order optimization method. In this case we don't need to explicitly know the matrix form of Jacobian `dh / dx`, but it's crucial for us to know the VJP of `dh / dx` with any input vector `v`. The way to get the VJPs in autograd is to use the `make_vjp` function.

In [60]:
from autograd import make_vjp
vjp_h_builder = make_vjp(h)

The `make_vjp(h)` function returns a builder function `vjp_L_builder(x)`. `vjp_L_builder(x)` takes in the same argument as `h`, and gives a tuple `(vjp_h_x_func(v), h)`. `vjp_h_x_func(v)` is yet another function, which takes in the "vector" of the vector-Jacobian product of `L` at `h` (i.e., `dL / dh`), and returns the VJP at `v`. 

**Another way to understand it is to think that `vjp_h_x_func` itself is the Jacobian matrix that gives `dh / dx`, and the input `v` is `dL / dh`, so that the output will be `dL / dv`.**

The ASCII diagram below illustrates the flow:
```
make_vjp(h) -> vjp_h(x) -> vjp_h_x(v) -> dL/dv
                              |    \_  
                              |      |
                            dh/dx * dL/dh (or any vector input)
                          Jacobian  Vector
```

In [61]:
vjp_h_x_func, h_value = vjp_h_builder(x)

Again, `vjp_h_x_func` is equivalent to Jacobian `dh / dx` (though we don't know, and don't have to know the full matrix of the Jacobian); if we pass the value of `dL / dh` to it, we should end up with the value of `dL / dx`:

In [62]:
print('dL / dx is ', vjp_h_x_func(J_L_h))

dL / dx is  [-7.36051353e+01 -2.02412671e+05]


Now what if we want to get `dL / df1` (whose value should be [-3.68025676e+01, -5.06031679e+04])? We need to get calculate the VJP of `dh / f1 * dL / dh`. For this, we need to construct another instance of `h` that takes `f1` as input:

In [63]:
h_f1 = lambda f1: f2(f1)

And then get the VJP builder function following the similar procedure above:

In [64]:
vjp_h_f1_builder = make_vjp(h_f1)
f1_value = f1(x)
vjp_h_f1_func, h_value = vjp_h_f1_builder(f1_value)

`vjp_h_f1_func` is essentially `dh / df1`. To get `dL / df1`, we pass to it `dL / dh`:

In [66]:
print('dL / df1 is ', vjp_h_f1_func(J_L_h))

dL / df1 is  [[-3.68025677e+01 -5.06031679e+04]]


Similarly, to get the VJP function `df1 / dx`:

In [73]:
vjp_f1_builder = make_vjp(f1)
vjp_f1_x_func, f1_value = vjp_f1_builder(x)

With all these, we can then formulate the backpropagation process, which should give the ultimate gradient `dL / dx` (supposed to be [-7.36051353e+01 -2.02412671e+05]) by nesting all these VJP functions:

In [76]:
J_L_x_explicit = vjp_f1_x_func(vjp_h_f1_func(J_L_h))
print('The ultimate gradient is ', J_L_x_explicit)

The ultimate gradient is  [-7.36051353e+01 -2.02412671e+05]


Of course we can get the same result using the `grad()` function, but building all the VJPs gives us an insight of how backpropagation functions behind the scene. 

### Hessians

Autograd provides a specific API for calculating the HVP through its `make_hvp` function, which behaves similarly to the `make_vjp` function. In fact, the `make_hvp(fun)` function is implemented simply as a callable that returns
```
make_vjp(grad(fun), x).
```
As noted previously, the Hessian of `L` with regards to `h` is a diagonal matrix `diag(2)`. Let us verify if the HVP of it with any vector `v` is `2v`.

In [77]:
from autograd.differential_operators import make_hvp
hvp_L_builder = make_hvp(L) # Note: L = L(h).
hvp_L_h_func, _ = hvp_L_builder(h(x), y)
print('HVP is [1, 2] is ', hvp_L_h_func([1, 2]))

HVP is [1, 2] is  [2. 4.]


## TensorFlow

TensorFlow's eager execution version has a `make_vjp` function which shares basically the same behavior with the one in Autograd, but if you don't use eager execution (which has poorer performance for large models), you need to build the VJP using `tf.gradient`. The good thing is that the `tf.gradient` function is able to give the gradient of `L` with regards to any node in the graph, so that we don't have to worry about what exactly is the input of the function we are differentiating. First, build the model using Tensors:

In [84]:
import tensorflow as tf

x = tf.constant([1., 2.], name='x')
y = tf.constant([9.48773584, 518.01282467], name='y')
f1 = tf.pow(x, 2, name='f1')
h = tf.exp(f1, name='h')
L = tf.reduce_sum(tf.squared_difference(h, y), name='L') 

sess = tf.Session()

Next, we attempt to reproduce all the VJPs we found above:

In [96]:
print('The values of dL/dh, dL/df1, and dL/dx are respectively:')
sess.run([tf.gradients(L, h),
          tf.gradients(L, f1),
          tf.gradients(L, x)])

The values of dL/dh, dL/df1, and dL/dx are respectively:


[[array([ -13.538908, -926.82935 ], dtype=float32)],
 [array([-3.6802567e+01, -5.0603168e+04], dtype=float32)],
 [array([-7.3605133e+01, -2.0241267e+05], dtype=float32)]]

If we ask TensorFlow to calculate the gradient of `h` with regards to `x`, which should be a Jacobian matrix since both `h` and `x` are vectors, it will in fact output the VJP of the Jacobian with an all-1 vector by default:

In [99]:
print('The value of dh/dx is ', sess.run(tf.gradients(h, x)))

The value of dh/dx is  [array([  5.4365635, 218.3926   ], dtype=float32)]


But what if we want to calculate the VJP with an arbitrary vector (i.e., how can we get a function for the VJP that can take in an arbitrary vector as input, as what we did with Autograd)? For example, the Jacobian `dh / df1` is
```
J_f2_f1 = [[exp(x1^2), 0        ],  = [[2.71828183, 0           ],
           [0        , exp(x2^2)]]     [0         , 54.59815003 ]].
```
If we explicitly use the Jacobian to calculate its product with `J_L_h = [-13.53890802, -926.82934927]`, we should end up with [-3.68025676e+01, -5.06031679e+04]. This should be done by passing the vector `v` to the `grad_ys` argument:

In [137]:
J_L_h = tf.constant([-13.53890802, -926.82934927])
vjp_h_f1_func = lambda v: tf.gradients(h, f1, grad_ys=[v])
print('VJP dh/df1 * dL/dh is', sess.run(vjp_h_f1_func(J_L_h)))
print('VJP dh/df1 * [1, 2] is', sess.run(vjp_h_f1_func([1., 2.])))

VJP dh/df1 * dL/dh is [array([-3.6802567e+01, -5.0603168e+04], dtype=float32)]
VJP dh/df1 * [1, 2] is [array([  2.7182817, 109.1963   ], dtype=float32)]


To compute the HVP, we can pass the gradient tensor to another `tf.gradients` function:

In [142]:
J_L_h_tensor = tf.gradients(L, h)
hvp_L_h_func = lambda v: tf.gradients(J_L_h_tensor, h, grad_ys=v)
print('HVP is [1, 2] is ', sess.run(hvp_L_h_func(tf.constant([1., 2.]))))

HVP is [1, 2] is  [array([2., 4.], dtype=float32)]


## PyTorch

Under construction.